In [1]:
print("OK")

OK


In [76]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle


In [3]:
# sep=";" specifies that columns in the CSV are separated by semicolons
# on_bad_lines="skip" tells Pandas to ignore rows that can't be parsed correctly
# encoding="latin-1" ensures compatibility with special characters in the file

books = pd.read_csv('BX-Books.csv', sep=";", on_bad_lines="skip", encoding='latin-1')


C:\Users\men_l\AppData\Local\Temp\ipykernel_27500\853055035.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('BX-Books.csv', sep=";", on_bad_lines="skip", encoding='latin-1')


In [4]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [5]:
# Access the large image URL from row 237 of the 'books' DataFrame
# .iloc[237] selects the 238th row (indexing starts at 0)
# ['Image-URL-L'] extracts the value from the 'Image-URL-L' column, which contains the URL of the large book cover image

books.iloc[237]['Image-URL-L']


'http://images.amazon.com/images/P/0671027387.01.LZZZZZZZ.jpg'

In [6]:
# Get the shape (dimensions) of the 'books' DataFrame
# books.shape returns a tuple (rows, columns)
# rows: total number of book entries in the dataset
# columns: total number of attributes/features about each book

books.shape


(271360, 8)

In [7]:
books.columns

Index(['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher',
       'Image-URL-S', 'Image-URL-M', 'Image-URL-L'],
      dtype='object')

In [8]:
# Select specific columns from the 'books' DataFrame to create a refined version
# This keeps only the most relevant details about each book, removing any unnecessary columns
# Columns included: ISBN, Title, Author, Year, Publisher, and URLs for different image sizes

books = books[['ISBN', 'Book-Title', 'Book-Author', 'Year-Of-Publication', 'Publisher', 'Image-URL-L']]


In [9]:
books.head()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [10]:
books.rename(columns={'Book-Title': 'title',  
                      'Book-Author': 'author',  
                      'Year-Of-Publication': 'year',  
                      'Publisher': 'publisher',  
                      'Image-URL-L': 'image_url'}, inplace=True)


In [11]:
books.head()

,ISBN,title,author,year,publisher,image_url
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...


In [12]:
# Load the BX-Users.csv file into a Pandas DataFrame
# sep=";" specifies that columns in the CSV are separated by semicolons
# on_bad_lines="skip" tells Pandas to ignore rows that can't be parsed correctly (recommended over deprecated error_bad_lines)
# encoding="latin-1" ensures proper handling of special characters in the file

users = pd.read_csv('BX-Users.csv', sep=";", on_bad_lines="skip", encoding='latin-1')


In [13]:
users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [14]:
users.shape

(278858, 3)

In [15]:
users.columns

Index(['User-ID', 'Location', 'Age'], dtype='object')

In [16]:
users.rename(columns={'User-ID': 'user_id', 
                      'Location': 'location', 
                      'Age': 'age'}, inplace=True)

In [17]:
users.head()

,user_id,location,age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [18]:
# Load the BX-Book-Ratings.csv file into a Pandas DataFrame
# sep=";" specifies that columns in the CSV are separated by semicolons
# on_bad_lines="skip" skips any rows that cannot be parsed correctly (recommended over the deprecated error_bad_lines)
# encoding="latin-1" ensures proper handling of special characters in the file

ratings = pd.read_csv('BX-Book-Ratings.csv', sep=";", on_bad_lines="skip", encoding='latin-1')


In [19]:
ratings.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [20]:
ratings.shape

(1149780, 3)

In [21]:
ratings.columns

Index(['User-ID', 'ISBN', 'Book-Rating'], dtype='object')

In [22]:
ratings.rename(columns={'User-ID': 'user_id',
                        'Book-Rating': 'rating'}, inplace=True)

In [23]:
ratings.head()

,user_id,ISBN,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [24]:
print(books.shape, users.shape, ratings.shape, sep='\n')

(271360, 6)
(278858, 3)
(1149780, 3)


In [25]:
# Count the number of ratings each user has submitted
# ratings['user_id'] selects the user_id column, which contains unique user identifiers
# .value_counts() counts how many times each user_id appears (i.e., how many ratings each user has given)
# The output is sorted by default, showing the most active users first

ratings['user_id'].value_counts()


user_id
11676     13602
198711     7550
153662     6109
98391      5891
35859      5850
          ...  
116180        1
116166        1
116154        1
116137        1
276723        1
Name: count, Length: 105283, dtype: int64

In [26]:
# Create a Boolean Series to identify users who have rated more than 200 books
# ratings['user_id'].value_counts() counts how many ratings each user has submitted
# > 200 checks if the user's rating count exceeds 200
# The result is True for users with more than 200 ratings and False otherwise

x = ratings['user_id'].value_counts() > 200


In [27]:
x

user_id
11676      True
198711     True
153662     True
98391      True
35859      True
          ...  
116180    False
116166    False
116154    False
116137    False
276723    False
Name: count, Length: 105283, dtype: bool

In [28]:
# Count the number of users who have rated more than 200 books
# x[x] filters the Series to include only users where x is True (i.e., those with >200 ratings)
# .shape returns the number of rows, indicating how many users meet this condition

x[x].shape


(899,)

In [29]:
# Get the user IDs of users who have rated more than 200 books
# x[x] filters the Series to keep only users with >200 ratings (where x is True)
# .index retrieves the user IDs of these highly active users

y = x[x].index


In [30]:
y

Index([ 11676, 198711, 153662,  98391,  35859, 212898, 278418,  76352, 110973,
       235105,
       ...
       260183,  73681,  44296, 155916,   9856, 274808,  28634,  59727, 268622,
       188951],
      dtype='int64', name='user_id', length=899)

In [31]:
# Filter the ratings DataFrame to include only users who have rated more than 200 books
# ratings['user_id'].isin(y) checks if each user_id is present in the list y (highly active users)
# ratings[...] selects only the rows where the condition is True, keeping only ratings from frequent reviewers

ratings = ratings[ratings['user_id'].isin(y)]


In [32]:
ratings.head()

,user_id,ISBN,rating
1456,277427,002542730X,10
1457,277427,0026217457,0
1458,277427,003008685X,8
1459,277427,0030615321,0
1460,277427,0060002050,0


In [33]:
# Merge the ratings DataFrame with the books DataFrame based on the 'ISBN' column
# This combines book details with user ratings, ensuring each rating now includes book information
# 'on="ISBN"' ensures the merge happens using the common column 'ISBN' that exists in both DataFrames

ratings_with_books = ratings.merge(books, on='ISBN')


In [34]:
ratings_with_books.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...


In [35]:
# Group the ratings_with_books DataFrame by the 'title' column to count the number of ratings for each book
# .groupby('title') organizes the data by book titles
# ['rating'].count() calculates how many ratings each book has received
# .reset_index() ensures the result is returned as a DataFrame instead of a grouped object

number_rating = ratings_with_books.groupby('title')['rating'].count().reset_index()


In [36]:
number_rating.head()

,title,rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [37]:
number_rating.rename(columns={'rating': 'number_of_rating'}, inplace=True)

In [38]:
number_rating.head()

,title,number_of_rating
0,A Light in the Storm: The Civil War Diary of ...,2
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,Beyond IBM: Leadership Marketing and Finance ...,1
4,Clifford Visita El Hospital (Clifford El Gran...,1


In [39]:
final_rating = ratings_with_books.merge(number_rating, on='title')

In [40]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
1,277427,0026217457,0,Vegetarian Times Complete Cookbook,Lucy Moll,1995,John Wiley &amp; Sons,http://images.amazon.com/images/P/0026217457.0...,7
2,277427,003008685X,8,Pioneers,James Fenimore Cooper,1974,Thomson Learning,http://images.amazon.com/images/P/003008685X.0...,1
3,277427,0030615321,0,"Ask for May, Settle for June (A Doonesbury book)",G. B. Trudeau,1982,Henry Holt &amp; Co,http://images.amazon.com/images/P/0030615321.0...,1
4,277427,0060002050,0,On a Wicked Dawn (Cynster Novels),Stephanie Laurens,2002,Avon Books,http://images.amazon.com/images/P/0060002050.0...,13


In [41]:
# Filter the final_rating DataFrame to include only books that have received at least 50 ratings
# final_rating['number_of_rating'] refers to the column that stores the count of ratings for each book
# >= 50 ensures we keep only books with 50 or more ratings, helping to focus on more popular books

final_rating = final_rating[final_rating['number_of_rating'] >= 50]


In [42]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [43]:
# Remove duplicate entries based on user_id and title to ensure unique ratings for each user-book pair
# drop_duplicates(['user_id', 'title']) checks for duplicate rows based on these columns
# inplace=True applies the changes directly to the DataFrame without creating a new copy

final_rating.drop_duplicates(['user_id', 'title'], inplace=True)


In [44]:
# Check for missing values in the final_rating DataFrame
# .isnull() identifies NaN (missing) values in the dataset
# .sum() counts the number of missing values in each column

final_rating.isnull().sum()


user_id             0
ISBN                0
rating              0
title               0
author              0
year                0
publisher           0
image_url           0
number_of_rating    0
dtype: int64

In [45]:
# Let's create a pivot table to reorganize the data. 
# This table will have book titles as the row index and user IDs as the column headers.
# The values in the table will be the ratings given by users to each book.
book_pivot = final_rating.pivot_table(columns='user_id', index='title', values='rating')


In [46]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
1st to Die: A Novel,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2nd Chance,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN
4 Blondes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84 Charing Cross Road,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,NaN,NaN,NaN,7.0,NaN,NaN,NaN,NaN,7.0,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
You Belong To Me,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN


In [47]:
book_pivot.shape

(742, 888)

In [48]:
# Replace any missing values (NaN) in the pivot table with 0.
# This ensures that there are no gaps in the dataset where a user hasn't rated a book.
# The 'inplace=True' parameter modifies the pivot table directly instead of creating a new copy.
book_pivot.fillna(0, inplace=True)


In [49]:
book_pivot

user_id,254,2276,2766,2977,3363,3757,4017,4385,6242,6251,...,274004,274061,274301,274308,274808,275970,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
84 Charing Cross Road,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Year of Wonders,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,7.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
You Belong To Me,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zen and the Art of Motorcycle Maintenance: An Inquiry into Values,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
# Convert the pivot table into a sparse matrix format using csr_matrix.
# This transformation helps optimize storage and computation when working with large datasets.
# Sparse matrices store only nonzero values, reducing memory usage and improving performance.
book_sparse = csr_matrix(book_pivot)


In [51]:
book_sparse

<742x888 sparse matrix of type '<class 'numpy.float64'>'
	with 14961 stored elements in Compressed Sparse Row format>

In [52]:
type(book_sparse)

scipy.sparse._csr.csr_matrix

In [53]:
# Create a Nearest Neighbors model using the brute-force search algorithm.
# The brute method computes distances between all points, making it effective for small to medium-sized datasets.
# This model will help find similar items based on proximity in the data.
model = NearestNeighbors(algorithm='brute')


In [54]:
# Train the Nearest Neighbors model using the sparse matrix representation of the dataset.
# This step allows the model to learn the structure of the data and prepare for similarity searches.
# After fitting, the model can efficiently find similar books based on user ratings.
model.fit(book_sparse)


NearestNeighbors(algorithm='brute')

In [55]:
# Select the 237th row from the pivot table 'book_pivot' and return all its columns.
# This retrieves the ratings given by various users for the book located at index 237.
# The result will be a single row representing the book's ratings across different users.
book_pivot.iloc[237, :]


user_id
254       9.0
2276      0.0
2766      0.0
2977      0.0
3363      0.0
         ... 
275970    9.0
277427    0.0
277478    0.0
277639    0.0
278418    0.0
Name: Harry Potter and the Chamber of Secrets (Book 2), Length: 888, dtype: float64

In [56]:
# Find the 6 nearest neighbors for the book at index 237 using the trained Nearest Neighbors model.
# The '.values.reshape(1, -1)' ensures the input is in the correct format for the model by converting it into a 2D array.
# The 'distance' variable stores the computed distances between the selected book and its nearest neighbors.
# The 'suggestion' variable contains the indices of the closest books in the dataset.
distance, suggestion = model.kneighbors(book_pivot.iloc[237, :].values.reshape(1, -1), n_neighbors=6)


In [57]:
distance

array([[ 0.        , 67.75691847, 68.05145112, 72.277244  , 75.81556568,
        76.30203143]])

In [58]:
suggestion

array([[237, 238, 240, 241, 184, 536]], dtype=int64)

In [59]:
# Iterate through the indices of the suggested nearest neighbors.
# 'suggestion' contains the indices of books that are most similar to the one at index 237.
# 'book_pivot.index[suggestion[i]]' retrieves the title of each suggested book.
# This loop prints the titles of the recommended books based on similarity.
for i in range(len(suggestion)):
    print(book_pivot.index[suggestion[i]])


Index(['Harry Potter and the Chamber of Secrets (Book 2)',
       'Harry Potter and the Goblet of Fire (Book 4)',
       'Harry Potter and the Prisoner of Azkaban (Book 3)',
       'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
       'The Cradle Will Fall'],
      dtype='object', name='title')


In [60]:
# Store the book titles from the pivot table index in 'book_names'.
# The index of 'book_pivot' contains the names of the books, which are extracted and stored for reference.
# This allows easy access to book names for further processing or recommendation tasks.
book_names = book_pivot.index


In [61]:
book_names[4]

'84 Charing Cross Road'

In [62]:
final_rating.head()

,user_id,ISBN,rating,title,author,year,publisher,image_url,number_of_rating
0,277427,002542730X,10,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner,1994,John Wiley &amp; Sons Inc,http://images.amazon.com/images/P/002542730X.0...,82
13,277427,0060930535,0,The Poisonwood Bible: A Novel,Barbara Kingsolver,1999,Perennial,http://images.amazon.com/images/P/0060930535.0...,133
15,277427,0060934417,0,Bel Canto: A Novel,Ann Patchett,2002,Perennial,http://images.amazon.com/images/P/0060934417.0...,108
18,277427,0061009059,9,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich,1995,HarperTorch,http://images.amazon.com/images/P/0061009059.0...,108
24,277427,006440188X,0,The Secret Garden,Frances Hodgson Burnett,1998,HarperTrophy,http://images.amazon.com/images/P/006440188X.0...,79


In [63]:
# Find the index position of the book titled '4 Blondes' in the pivot table.
# 'np.where(book_pivot.index == '4 Blondes')' returns an array of indices where the condition is met.
# '[0][0]' extracts the first occurrence, giving the exact row index of '4 Blondes' in the dataset.
np.where(book_pivot.index == '4 Blondes')[0][0]


3

In [66]:
# Locate the index of the book titled "Harry Potter and the Chamber of Secrets (Book 2)" in the 'final_rating' dataset.
# 'np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")' searches for the row where the title matches.
# '[0][0]' extracts the first occurrence, returning the exact row index of the book in the dataset.
ids = np.where(final_rating['title'] == "Harry Potter and the Chamber of Secrets (Book 2)")[0][0]


In [68]:
final_rating.iloc[ids]['image_url']

'http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg'

In [69]:
# Initialize an empty list to store book names
book_name = []

# Iterate over each book_id in the suggestion list
for book_id in suggestion:
    # Append the book name corresponding to the book_id from the book_pivot index
    book_name.append(book_pivot.index[book_id])


In [70]:
book_name

[Index(['Harry Potter and the Chamber of Secrets (Book 2)',
        'Harry Potter and the Goblet of Fire (Book 4)',
        'Harry Potter and the Prisoner of Azkaban (Book 3)',
        'Harry Potter and the Sorcerer's Stone (Book 1)', 'Exclusive',
        'The Cradle Will Fall'],
       dtype='object', name='title')]

In [72]:
# Initialize an empty list to store the index values of book IDs
ids_index = []

# Iterate over each book name in the first element of book_name list
for name in book_name[0]:
    # Find the index where the book title in final_rating matches the current book name
    ids = np.where(final_rating['title'] == name)[0][0]
    
    # Append the found index to the ids_index list
    ids_index.append(ids)


In [73]:
ids_index

[44, 321, 45, 46, 786, 2297]

In [74]:
# Iterate over each index in the ids_index list
for idx in ids_index:
    # Retrieve the image URL corresponding to the current index from final_rating DataFrame
    url = final_rating.iloc[idx]['image_url']
    
    # Print the extracted image URL
    print(url)


http://images.amazon.com/images/P/0439064872.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439139597.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0439136369.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/043936213X.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0446604232.01.LZZZZZZZ.jpg
http://images.amazon.com/images/P/0440115450.01.LZZZZZZZ.jpg


In [79]:
import pickle  # Import the pickle module for saving objects

# Save the trained model to a file named 'model.pk1' in the 'artifacts' directory
pickle.dump(model, open('artifacts/model.pk1', 'wb'))

# Save the list of book names to a file named 'book_names.pk1' in the 'artifacts' directory
pickle.dump(book_names, open('artifacts/book_names.pk1', 'wb'))

# Save the final_rating DataFrame to a file named 'final_rating.pk1' in the 'artifacts' directory
pickle.dump(final_rating, open('artifacts/final_rating.pk1', 'wb'))

# Save the book_pivot DataFrame to a file named 'book_pivort.pk1' in the 'artifacts' directory
# (Note: There seems to be a typo in 'book_pivort'. If it's meant to be 'book_pivot', consider correcting it.)
pickle.dump(book_pivot, open('artifacts/book_pivort.pk1', 'wb'))


## Testing Model 

In [81]:
def recommend_book(book_name):
    # Find the index of the given book_name in the book_pivot index
    book_id = np.where(book_pivot.index == book_name)[0][0]

    # Use the trained model to find the 6 nearest neighbors (similar books)
    distance, suggestion = model.kneighbors(book_pivot.iloc[book_id, :].values.reshape(1, -1), n_neighbors=6)

    # Iterate over the suggested book indices
    for i in range(len(suggestion)):
        # Retrieve the book names corresponding to the suggested indices
        books = book_pivot.index[suggestion[i]]

        # Iterate over each book in the suggested books list
        for j in books:
            # If the book is the one searched for, display a heading message
            if j == book_name:
                print(f"You searched for '{book_name}'\n")
                print("The suggested books are:\n")
            else:
                # Print the suggested book name
                print(j)


In [82]:
# Define the book name to be searched
book_name = "Harry Potter and the Chamber of Secrets (Book 2)"

# Call the recommend_book function to find similar book recommendations
recommend_book(book_name)


You searched for 'Harry Potter and the Chamber of Secrets (Book 2)'

The suggested books are:

Harry Potter and the Goblet of Fire (Book 4)
Harry Potter and the Prisoner of Azkaban (Book 3)
Harry Potter and the Sorcerer's Stone (Book 1)
Exclusive
The Cradle Will Fall


In [84]:
# Define the book name to be searched
book_name = "2nd Chance"

# Call the recommend_book function to find similar book recommendations
recommend_book(book_name)


You searched for '2nd Chance'

The suggested books are:

The Next Accident
The Ghost
Exclusive
Last Man Standing
Unspeakable
